In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


In [18]:
filePath = 'data/data.xlsx'
df_var = pd.read_excel(filePath, header=None, nrows=2)
variable_types = df_var.iloc[0]
variable = df_var.iloc[1][variable_types == "x"].tolist()
df = pd.read_excel(filePath, header=1,)

In [19]:
data = df[variable]

In [20]:
X = data  # Remplacez par vos attributs

# Normaliser les données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 2. Appliquer K-means
k = 3  # Nombre de clusters souhaité
kmeans = KMeans(n_clusters=k, random_state=42)
data['cluster'] = kmeans.fit_predict(X_scaled)



/tmp/ipykernel_8427/3992460051.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['cluster'] = kmeans.fit_predict(X_scaled)


In [21]:
results = {}

global_means = data.mean()
global_std = data.std()
n = data.shape[0]

for i in range(k):
    result = {} 
    data_cluster = data[data['cluster'] == i]
    n_C = data_cluster.shape[0]
    for var in variable:
        var_results = {}  
        mean_cluster = data_cluster[var].mean()
        global_mean = global_means[var]
        std_dev = global_std[var]
        test_value = (mean_cluster - global_mean) / std_dev * np.sqrt(n / n_C)
        # values = pd.unique(data_cluster[var]).tolist()
        # for value in values:
        mean = data_cluster[var].mean()  
            # var_results[value] = mean
        result[var] = {"test_value": test_value, "mean": mean , "Global mean": global_mean}
        
    results[f"Cluster {i}"] = result

# `results` contiendra un dictionnaire avec les moyennes de chaque variable et valeur pour chaque cluster


/tmp/ipykernel_8427/374192162.py:16: RuntimeWarning: invalid value encountered in scalar divide
  test_value = (mean_cluster - global_mean) / std_dev * np.sqrt(n / n_C)


In [23]:
# Initialisation d'une liste pour stocker les données structurées
data_list = []

# Parcours du dictionnaire `results` pour construire les lignes du DataFrame
for cluster_name, features in results.items():
    for feature, values in features.items():
        # Ajout d'une ligne pour chaque combinaison cluster-feature avec les valeurs correspondantes
        data_list.append({
            'Cluster': cluster_name,
            'Feature': feature,
            'test_value': values['test_value'].round(2),
            'mean': values['mean'].round(2),
            'global_mean': values['Global mean'].round(2)
        })

# Conversion de la liste en DataFrame
df_ = pd.DataFrame(data_list)


# Affichage du DataFrame
df_

,Cluster,Feature,test_value,mean,global_mean
0,Cluster 0,P3. Patient age:,-0.67,67.75,71.51
1,Cluster 0,P21bP21aP21dP21cR.Duration of treatment in pre...,0.47,12.05,9.86
2,Cluster 0,P22bP21dP22bP21bR.Number of months between end...,0.18,2.50,2.06
3,Cluster 0,P5b2. Does this patient have any of the follow...,0.49,0.16,0.08
4,Cluster 0,P5b2. Does this patient have any of the follow...,0.36,0.19,0.12
...,...,...,...,...,...
310,Cluster 2,P6R3. RI status - No RI,-1.89,0.15,0.57
311,Cluster 2,P6R4. Comorbidities Vs No Comorbidities - No c...,-0.87,0.00,0.13
312,Cluster 2,P6R4. Comorbidities Vs No Comorbidities - At l...,0.87,1.00,0.87
313,Cluster 2,P7bR1. Current transplant status (simplified) ...,-1.44,0.05,0.36


In [ ]:
def calculate_kmeans(df, n_clusters):
    
    X = df  # Remplacez par vos attributs

    # Normaliser les données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # 2. Appliquer K-means
    k = 3  # Nombre de clusters souhaité
    kmeans = KMeans(n_clusters=k, random_state=42)
    df['cluster'] = kmeans.fit_predict(X_scaled)
    
    results = {}

    global_means = df.mean()
    global_std = df.std()
    n = df.shape[0]

    for i in range(k):
        result = {} 
        data_cluster = df[df['cluster'] == i]
        n_C = data_cluster.shape[0]
        for var in df.columns:
            var_results = {}  
            mean_cluster = data_cluster[var].mean()
            global_mean = global_means[var]
            std_dev = global_std[var]
            test_value = (mean_cluster - global_mean) / std_dev * np.sqrt(n / n_C)
            # values = pd.unique(data_cluster[var]).tolist()
            # for value in values:
            mean = data_cluster[var].mean()  
                # var_results[value] = mean
            result[var] = {"test_value": test_value, "mean": mean , "Global mean": global_mean}
            
        results[f"Cluster {i}"] = result
    
    data_list = []

    # Parcours du dictionnaire `results` pour construire les lignes du DataFrame
    for cluster_name, features in results.items():
        for feature, values in features.items():
            # Ajout d'une ligne pour chaque combinaison cluster-feature avec les valeurs correspondantes
            data_list.append({
                'Cluster': cluster_name,
                'Feature': feature,
                'test_value': values['test_value'].round(2),
                'mean': values['mean'].round(2),
                'global_mean': values['Global mean'].round(2)
            })

    # Conversion de la liste en DataFrame
    df_ = pd.DataFrame(data_list)
    return df_